<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill for Air Quality Data</span>


## 🗒️ You have the following tasks
1. Choose an Air Quality Sensor
2. Update the country, city, and street information to point to YOUR chosen Air Quality Sensor
3. Download historical measures for your Air Quality Sensor as a CSV file
4. Update the path of the CSV file in this notebook to point to the one that you downloaded
5. Create an account on www.hopsworks.ai and get your HOPSWORKS_API_KEY
6. Run this notebook



### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
with open('../data/keys/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
    print(os.environ["HOPSWORKS_API_KEY"])
# logout from hopsworks first
project = hopsworks.login(project="ML_Project_Electricity", api_key_value=os.environ["HOPSWORKS_API_KEY"])
print(f"Project: {project.name}")

# util.purge_project(proj)


x5PZCK0patagTBDP.SjxqfFY5JlMcD4HCF4BEd0rDdMJVabM44KJEdZRIVrYSoxvGd8lfgz23KgblRcI9
2025-01-08 14:08:26,669 INFO: Initializing external client
2025-01-08 14:08:26,670 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 14:08:26,670 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 14:08:27,805 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207495
Project: ML_Project_Electricity


## LOAD ALL THE DATA

In [3]:
# Start by loading historical electricity prices:
# Load the Elspotprices.csv file
csv_file = Path("../data/Elspotprices.csv")
e_df = pd.read_csv(csv_file, delimiter=";")
e_df.head()

# Remove HourUTC and SpotpriceDKK column
e_df = e_df.drop(columns=['HourUTC', 'SpotPriceDKK'])

# Rename HourDK to Time and make it proper datetime format
e_df = e_df.rename(columns={"HourDK": "time"})
e_df['time'] = pd.to_datetime(e_df['time'])

# Make the Time column the index of the DataFrame
# e_df = e_df.set_index('time')

# Format SpotPriceEUR into a float
e_df['SpotPriceEUR'] = e_df['SpotPriceEUR'].str.replace(',', '.').astype(float)

# Split the DataFrame based on the 'PriceArea' column
price_area_groups = e_df.groupby('PriceArea')

# Create separate DataFrames for each PriceArea
price_area_dfs = {price_area: group for price_area, group in price_area_groups}

# Access individual DataFrames
se3_df = price_area_dfs['SE3']
se4_df = price_area_dfs['SE4']

se3_df.head()

,time,PriceArea,SpotPriceEUR
0,2024-12-31 23:00:00,SE3,2.24
3,2024-12-31 22:00:00,SE3,7.93
4,2024-12-31 21:00:00,SE3,14.98
7,2024-12-31 20:00:00,SE3,21.51
8,2024-12-31 19:00:00,SE3,23.01


In [5]:
# add a rolling mean of the last 24*4 hours to the price data
# Sort the data by time before doing the rolling mean
se3_df = se3_df.sort_values('time')
se4_df = se4_df.sort_values('time')

se3_df['spot_price_rolling'] = se3_df['SpotPriceEUR'].rolling(window=24*7).mean()
se4_df['spot_price_rolling'] = se4_df['SpotPriceEUR'].rolling(window=24*7).mean()

# Add a rolling mean of the last 3 hours
se3_df['spot_price_rolling_3h'] = se3_df['SpotPriceEUR'].rolling(window=3).mean()
se4_df['spot_price_rolling_3h'] = se4_df['SpotPriceEUR'].rolling(window=3).mean()

se4_df.head(3)


# order needs to be reversed?

,time,PriceArea,SpotPriceEUR,spot_price_rolling,spot_price_rolling_3h
35087,2023-01-01 00:00:00,SE4,2.01,NaN,NaN
35085,2023-01-01 01:00:00,SE4,1.38,NaN,NaN
35083,2023-01-01 02:00:00,SE4,0.09,NaN,1.16


In [7]:
# Load the weather data for Stockholm and Malmo

weather_sthlm_df = pd.read_csv("../data/Hourly_weather_stockholm.csv", header=2)
weather_malmo_df = pd.read_csv("../data/Hourly_weather_malmo.csv", header=2)

# weather_sthlm_df.head()

# Splite the 'time' column into 'Date', 'Weekday', 'Month' and 'Hour' columns
weather_sthlm_df['time'] = pd.to_datetime(weather_sthlm_df['time'])
weather_sthlm_df['Date'] = weather_sthlm_df['time'].dt.date
weather_sthlm_df['Weekday'] = weather_sthlm_df['time'].dt.weekday
weather_sthlm_df['Month'] = weather_sthlm_df['time'].dt.month
weather_sthlm_df['Hour'] = weather_sthlm_df['time'].dt.hour

weather_malmo_df['time'] = pd.to_datetime(weather_malmo_df['time'])
weather_malmo_df['Date'] = weather_malmo_df['time'].dt.date
weather_malmo_df['Weekday'] = weather_malmo_df['time'].dt.weekday
weather_malmo_df['Month'] = weather_malmo_df['time'].dt.month
weather_malmo_df['Hour'] = weather_malmo_df['time'].dt.hour

weather_sthlm_df.head()

,time,temperature,precipitation,cloud_cover,wind_speed_10m,Date,Weekday,Month,Hour
0,2023-01-01 00:00:00,2.8,0.0,87,15.3,2023-01-01,6,1,0
1,2023-01-01 01:00:00,2.4,0.0,100,14.8,2023-01-01,6,1,1
2,2023-01-01 02:00:00,1.6,0.0,100,11.2,2023-01-01,6,1,2
3,2023-01-01 03:00:00,0.8,0.0,100,10.4,2023-01-01,6,1,3
4,2023-01-01 04:00:00,0.2,0.0,100,9.7,2023-01-01,6,1,4


In [8]:
# Combine the hourly weather data with daily total sunshine duration data

# Load the daily sunshine duration data for SE3 (Stockholm)
sunshine_stockholm_df = pd.read_csv("../data/Daily_sunshine_stockholm.csv", header=2)
sunshine_stockholm_df['Date'] = pd.to_datetime(sunshine_stockholm_df['time'])
sunshine_stockholm_df = sunshine_stockholm_df.drop(columns=['time'])
sunshine_stockholm_df = sunshine_stockholm_df.set_index('Date')

# Merge the weather data (weather_sthlm_df) with the sunshine duration data (if it hasn't been done already)
if 'sunshine_duration' not in weather_sthlm_df.columns:
    # Merge the weather data with the sunshine duration data
    weather_sthlm_df['Date'] = weather_sthlm_df['time'].dt.date
    weather_sthlm_df = weather_sthlm_df.set_index('Date')
    weather_sthlm_df = weather_sthlm_df.join(sunshine_stockholm_df, how='left')
    weather_sthlm_df = weather_sthlm_df.reset_index()
    print("Merge completed.")
else:
    print("Merge has already been done.")

weather_sthlm_df.head()

# Load the daily sunshine duration data for SE4 (Malmö)
sunshine_malmo_df = pd.read_csv("../data/Daily_sunshine_malmo.csv", header=2)
sunshine_malmo_df['Date'] = pd.to_datetime(sunshine_malmo_df['time'])
sunshine_malmo_df = sunshine_malmo_df.drop(columns=['time'])
sunshine_malmo_df = sunshine_malmo_df.set_index('Date')

# sunshine_malmo_df.head()

# Merge the weather data with the sunshine duration data (if it hasn't been done already)
if 'sunshine_duration' not in weather_malmo_df.columns:
    # Merge the weather data with the sunshine duration data
    weather_malmo_df['Date'] = weather_malmo_df['time'].dt.date
    weather_malmo_df = weather_malmo_df.set_index('Date')
    weather_malmo_df = weather_malmo_df.join(sunshine_malmo_df, how='left')
    weather_malmo_df = weather_malmo_df.reset_index()
    print("Merge completed.")
else:
    print("Merge has already been done.")

weather_sthlm_df.head()

Merge completed.
Merge completed.


,Date,time,temperature,precipitation,cloud_cover,wind_speed_10m,Weekday,Month,Hour,sunshine_duration
0,2023-01-01,2023-01-01 00:00:00,2.8,0.0,87,15.3,6,1,0,13654.35
1,2023-01-01,2023-01-01 01:00:00,2.4,0.0,100,14.8,6,1,1,13654.35
2,2023-01-01,2023-01-01 02:00:00,1.6,0.0,100,11.2,6,1,2,13654.35
3,2023-01-01,2023-01-01 03:00:00,0.8,0.0,100,10.4,6,1,3,13654.35
4,2023-01-01,2023-01-01 04:00:00,0.2,0.0,100,9.7,6,1,4,13654.35


## DATA CLEANING

In [9]:
# Drop any rows with missing values
weather_sthlm_df = weather_sthlm_df.dropna()
weather_malmo_df = weather_malmo_df.dropna()

## Create feature groups

In [13]:
fs = project.get_feature_store()

# Create a feature group for the electricity prices in SE3
se3_fg = fs.get_or_create_feature_group(
    name="se3_electricity_prices", 
    version=1, 
    description="Electricity prices in SE3",
    primary_key=["time"],
    event_time="time")

# Create a feature group for the electricity prices in SE4
se4_fg = fs.get_or_create_feature_group(
    name="se4_electricity_prices", 
    version=1, 
    description="Electricity prices in SE4",
    primary_key=["time"],
    event_time="time")

# Create a feature group for the weather data in Stockholm
sthlm_fg = fs.get_or_create_feature_group(
    name="stockholm_weather", 
    version=1, 
    description="Weather data in Stockholm",
    primary_key=["time"],
    event_time="time")

# # Create a feature group for the weather data in Malmö
malmo_fg = fs.get_or_create_feature_group(
    name="malmo_weather", 
    version=1, 
    description="Weather data in Malmö",
    primary_key=["time"],
    event_time="time")



## Insert Historical data into feature groups

In [14]:
#Insert data into the feature groups
se3_fg.insert(se3_df)
se4_fg.insert(se4_df)
# sthlm_fg.insert(weather_sthlm_df)
# malmo_fg.insert(weather_malmo_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207495/fs/1195127/fg/1393774


Uploading Dataframe: 100.00% |██████████| Rows 17544/17544 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: se3_electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207495/jobs/named/se3_electricity_prices_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207495/fs/1195127/fg/1393775


Uploading Dataframe: 100.00% |██████████| Rows 17544/17544 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: se4_electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207495/jobs/named/se4_electricity_prices_1_offline_fg_materialization/executions


(Job('se4_electricity_prices_1_offline_fg_materialization', 'SPARK'), None)

## <span style='color:#ff5f27'> 🌍 STEP 5: Read your CSV file into a DataFrame </span>

The cell below will read up historical air quality data as a CSV file into a Pandas DataFrame

In [5]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df

,date,pm25,o3,so2,co
0,2024-11-01,15.0,27.0,NaN,1.0
1,2024-11-02,22.0,19.0,NaN,NaN
2,2024-11-03,23.0,22.0,NaN,NaN
3,2024-11-04,18.0,23.0,NaN,NaN
4,2024-11-05,17.0,19.0,NaN,NaN
...,...,...,...,...,...
3467,2015-06-29,NaN,42.0,1.0,1.0
3468,2015-06-30,NaN,33.0,2.0,1.0
3469,2015-01-15,NaN,11.0,NaN,1.0
3470,2015-01-26,NaN,15.0,NaN,1.0


## <span style='color:#ff5f27'> 🌍 STEP 6: Data cleaning + BONUS: add 3 day rolling mean of pm25</span>


### Rename columns if needed and drop unneccessary columns

We want to have a DataFrame with 2 columns - `date` and `pm25` after this cell below:

## Check the data types for the columns in your DataFrame

 * `date` should be of type   datetime64[ns] 
 * `pm25` should be of type float64

In [13]:
# These commands will succeed if your CSV file didn't have a `median` or `timestamp` column
df = df.rename(columns={"median": "pm25"})
df = df.rename(columns={"timestamp": "date"})

df_aq = df[['date', 'pm25']]

# Set the index to the 'date' column
# df_aq.set_index('date', inplace=True)

df_aq['pm25'] = df_aq['pm25'].astype('float32')

# Compute the rolling mean of the last 3 days, allowing for NaN values
df_aq['rolling_mean_pm25'] = df_aq['pm25'].rolling(window=3, min_periods=1).mean()

df_aq

,date,pm25,rolling_mean_pm25
0,2024-11-01,15.0,15.000000
1,2024-11-02,22.0,18.500000
2,2024-11-03,23.0,20.000000
3,2024-11-04,18.0,21.000000
4,2024-11-05,17.0,19.333333
...,...,...,...
3467,2015-06-29,NaN,NaN
3468,2015-06-30,NaN,NaN
3469,2015-01-15,NaN,NaN
3470,2015-01-26,NaN,NaN


In [14]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3472 entries, 0 to 3471
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               3472 non-null   datetime64[ns]
 1   pm25               3367 non-null   float32       
 2   rolling_mean_pm25  3369 non-null   float64       
dtypes: datetime64[ns](1), float32(1), float64(1)
memory usage: 67.9 KB


## <span style='color:#ff5f27'> 🌍 STEP 7: Drop any rows with missing data </span>
It will make the model training easier if there is no missing data in the rows, so we drop any rows with missing data.

In [15]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25,rolling_mean_pm25
0,2024-11-01,15.0,15.000000
1,2024-11-02,22.0,18.500000
2,2024-11-03,23.0,20.000000
3,2024-11-04,18.0,21.000000
4,2024-11-05,17.0,19.333333
...,...,...,...
3362,2015-03-27,21.0,36.000000
3363,2015-03-28,28.0,26.666667
3364,2015-03-29,34.0,27.666667
3365,2015-03-30,26.0,29.333333


## <span style='color:#ff5f27'> 🌍 STEP 8: Add country, city, street, url to the DataFrame </span>

Your CSV file may have many other air quality measurement columns. We will only work with the `pm25` column.

We add the columns for the country, city, and street names that you changed for your Air Quality sensor.

We also want to make sure the `pm25` column is a float32 data type.

In [16]:
# Your sensor may have columns we won't use, so only keep the date and pm25 columns
# If the column names in your DataFrame are different, rename your columns to `date` and `pm25`
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url
df_aq

,date,pm25,rolling_mean_pm25,country,city,street,url
0,2024-11-01,15.0,15.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
1,2024-11-02,22.0,18.500000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
2,2024-11-03,23.0,20.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3,2024-11-04,18.0,21.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
4,2024-11-05,17.0,19.333333,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
...,...,...,...,...,...,...,...
3362,2015-03-27,21.0,36.000000,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3363,2015-03-28,28.0,26.666667,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3364,2015-03-29,34.0,27.666667,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638
3365,2015-03-30,26.0,29.333333,usa,bondville,bondville-illinois-usa,https://api.waqi.info/feed/@7638


---

## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

## <span style='color:#ff5f27'> 🌍 STEP 9: Download the Historical Weather Data </span>

https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant

We will download the historical weather data for your `city` by first extracting the earliest date from your DataFrame containing the historical air quality measurements.

We will download all daily historical weather data measurements for your `city` from the earliest date in your air quality measurement DataFrame. It doesn't matter if there are missing days of air quality measurements. We can store all of the daily weather measurements, and when we build our training dataset, we will join up the air quality measurements for a given day to its weather features for that day. 

The weather features we will download are:

 * `temperature (average over the day)`
 * `precipitation (the total over the day)`
 * `wind speed (average over the day)`
 * `wind direction (the most dominant direction over the day)`


In [17]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

Coordinates 40.105445861816406°N -88.361328125°E
Elevation 221.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [18]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3576 entries, 0 to 3575
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         3576 non-null   datetime64[ns]
 1   temperature_2m_mean          3576 non-null   float32       
 2   precipitation_sum            3576 non-null   float32       
 3   wind_speed_10m_max           3576 non-null   float32       
 4   wind_direction_10m_dominant  3576 non-null   float32       
 5   city                         3576 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 139.7+ KB


## <span style='color:#ff5f27'> 🌍 STEP 10: Define Data Validation Rules </span>

We will validate the air quality measurements (`pm25` values) before we write them to Hopsworks.

We define a data validation rule (an expectation in Great Expectations) that ensures that `pm25` values are not negative or above the max value available by the sensor.

We will attach this expectation to the air quality feature group, so that we validate the `pm25` data every time we write a DataFrame to the feature group. We want to prevent garbage-in, garbage-out.

In [19]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

## Expectations for Weather Data
Here, we define an expectation for 2 columns in our weather DataFrame - `precipitation_sum` and `wind_speed_10m_max`, where we expect both values to be greater than zero, but less than 1000.

In [20]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

---

### <span style="color:#ff5f27;"> 🔮 STEP 11: Connect to Hopsworks and save the sensor country, city, street names as a secret</span>

In [26]:
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [27]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.
{"country": "usa", "city": "bondville", "street": "bondville-illinois-usa", "aqicn_url": "https://api.waqi.info/feed/@7638", "latitude": 40.11, "longitude": -88.37}


### <span style="color:#ff5f27;"> 🔮 STEP 12: Create the Feature Groups and insert the DataFrames in them </span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each air quality sensor measurement is uniquely identified by `country`, `street`, and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [31]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality_new',
    description='Air Quality characteristics of each day, with rolling mean of pm25',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

#### Insert the DataFrame into the Feature Group

In [32]:
air_quality_fg.insert(df_aq)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1161369/fs/1152072/fg/1347992
2024-11-14 18:34:41,086 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161369/fs/1152072/fg/1347992


Uploading Dataframe: 0.00% |          | Rows 0/3367 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_new_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161369/jobs/named/air_quality_new_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x15af0a770>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 671795
         }
       },
       "result": {
         "observed_value": 3.0,
         "element_count": 3367,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-14T05:34:41.000086Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

#### Enter a description for each feature in the Feature Group

In [33]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")
air_quality_fg.update_feature_description("rolling_mean_pm25", "Rolling mean of pm25 over the last 3 days")

### <span style='color:#ff5f27'> 🌦 Weather Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each weather measurement is uniquely identified by `city` and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [29]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

#### Insert the DataFrame into the Feature Group

In [30]:
# Insert data
weather_fg.insert(weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1161369/fs/1152072/fg/1342814
2024-11-08 18:36:47,575 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161369/fs/1152072/fg/1342814


Uploading Dataframe: 0.00% |          | Rows 0/3570 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161369/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x166765ed0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 666640
         }
       },
       "result": {
         "observed_value": 6.193674087524414,
         "element_count": 3570,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-08T05:36:47.000575Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

#### Enter a description for each feature in the Feature Group

In [31]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02: Daily Feature Pipeline 
 </span> 


## <span style="color:#ff5f27;">⏭️ **Exercises:** 
 </span> 

Extra Homework:

  * Try adding a new feature based on a rolling window of 3 days for 'pm25'
      * This is not easy, as forecasting more than 1 day in the future, you won't have the previous 3 days of pm25 measurements.
      * df.set_index("date").rolling(3).mean() is only the start....
  * Parameterize the notebook, so that you can provide the `country`/`street`/`city`/`url`/`csv_file` as parameters. 
      * Hint: this will also require making the secret name (`SENSOR_LOCATION_JSON`), e.g., add the street name as part of the secret name. Then you have to pass that secret name as a parameter when running the operational feature pipeline and batch inference pipelines.
      * After you have done this, collect the street/city/url/csv files for all the sensors in your city or region and you make dashboards for all of the air quality sensors in your city/region. You could even then add a dashboard for your city/region, as done [here for Poland](https://github.com/erno98/ID2223).

Improve this AI System
  * As of mid 2024, there is no API call available to download historical data from the AQIN website. You could improve this system by writing a PR to download the CSV file using Python Selenium and the URL for the sensor.


---